### Chroma

    - 오픈 소스 벡터 데이터베이스
    - 개발자의 생산성과 행복에 초점을 맞춤

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
from langchain_community.document_loaders import TextLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 600, chunk_overlap = 0)

# 텍스트 파일 로드
loader1 = TextLoader("./data/nlp-keywords.txt")
loader2 = TextLoader("./data/finance-keywords.txt")

# 문서 분할
split_doc1 = loader1.load_and_split(text_splitter)
split_doc2 = loader2.load_and_split(text_splitter)

print(len(split_doc1))
print(len(split_doc2))

11
6


### VectorStore 생성

**from_documents()**

    문서 리스트로부터 벡터 저장소를 생성한다.

**매개변수**

    documents : 벡터 저장소에 추가할 문서 리스트 (필수)
    collection_name : 생성할 컬렉션 이름 (필수)
    embeddings : 임베딩 함수
    ids : 문서 id 리스트
    persist_directory : 컬렉션 저장할 디렉토리
    client_settings : Chroma 사용자 설정
    client : 사용자 인스턴스
    collection_metadata : 컬렉션 구성 정보

In [6]:
db = Chroma.from_documents(
    documents=split_doc1, embedding=OpenAIEmbeddings(), collection_name="my_db"
)

In [14]:
# 저장 경로
DB_PATH = "./chroma_db"

persist_db = Chroma.from_documents(
    documents=split_doc1,
    embedding=OpenAIEmbeddings(),
    collection_name="my_db",
    persist_directory=DB_PATH
)

**from_texts**

    텍스트 리스트로 벡터 저장소를 생성

**매개변수**

    - texts : 벡터 저장소에 추가할 텍스트 리스트 (필수)
    - collection_name : 생성할 컬렉션 이름
    (기본값: LANGCHAIN_DEFAULT_COLLECTION_NAME)
    - embeddings : 임베딩 함수
    - ids : 문서 id 리스트
    - persist_directory : 컬렉션을 저장할 디렉토리
    - client_settings : Chroma 사용자 설정
    - client : 사용자 인스턴스
    - collection_metadata : 컬렉션 구성 정보


In [11]:
db2 = Chroma.from_texts(
    ["안녕하세요. 반갑습니다. 오늘 메뉴는 뭔가요?", "제 이름은 bear입니다."],
    embedding=OpenAIEmbeddings()
)

In [12]:
db2.get()

{'ids': ['831d2563-ff51-4f5d-8547-3bd20aeda0b6',
  '7000a050-fe7c-4665-8c81-90deec46dcd0'],
 'embeddings': None,
 'documents': ['안녕하세요. 반갑습니다. 오늘 메뉴는 뭔가요?', '제 이름은 bear입니다.'],
 'uris': None,
 'data': None,
 'metadatas': [None, None],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

**similarity_search**

    Chroma 데이터베이스에서 유사도 검색을 수행한다.
    주어진 쿼리와 가장 유사한 문서를 반환한다.

매개변수

    query : 검색할 쿼리 텍스트
    k : 반환할 결과의 수
    filter : 메타데이터로 필터링, 기본값 None

참고

    리턴 시 점수 정보 없이 문서만 반환이 된다. 점수 정보를 얻고 싶으면 (similarity_search_with_score())

In [15]:
db.similarity_search("csv에 대해 알려줘")

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


[Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: 토크나이저는 텍스트 데이터를 토큰으로 분할하는 도구입니다. 이는 자연어 처리에서 데이터를 전처리하는 데 사용됩니다.\n예시: "I love programming."이라는 문장을 ["I", "love", "programming", "."]으로 분할합니다.\n연관키워드: 토큰화, 자연어 처리, 구문 분석\n\nVectorStore\n\n정의: 벡터스토어는 벡터 형식으로 변환된 데이터를 저장하는 시스템입니다. 이는 검색, 분류 및 기타 데이터 분석 작업에 사용됩니다.\n예시: 단어 임베딩 벡터들을 데이터베이스에 저장하여 빠르게 접근할 수 있습니다.\n연관키워드: 임베딩, 데이터베이스, 벡터화\n\nSQL\n\n정의: SQL(Structured Query Language)은 데이터베이스에서 데이터를 관리하기 위한 프로그래밍 언어입니다. 데이터 조회, 수정, 삽입, 삭제 등 다양한 작업을 수행할 수 있습니다.\n예시: SELECT * FROM users WHERE age > 18;은 18세 이상의 사용자 정보를 조회합니다.\n연관키워드: 데이터베이스, 쿼리, 데이터 관리\n\nCSV'),
 Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='Semantic Search\n\n정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.\n예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.\n연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝\n\nEmbedding\n\n정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.\

**add_documents()**

    벡터 스토어에 문서 추가를 할 수 있다.

매개변수

    documents : 벡터 저장소에 추가할 문서 리스트
    **kwargs : 추가 키워드 인자
        ids : 문서 ID 리스트

In [16]:
from langchain_core.documents import Document

db.add_documents(
    [
        Document(
            page_content="안녕하세요! 이번에는 문서를 추가해볼거예요",
            metadata={"source" : "mydata.txt"},
            id="1"
        )
    ]
)

['1']

In [17]:
db.get("1")

{'ids': ['1'],
 'embeddings': None,
 'documents': ['안녕하세요! 이번에는 문서를 추가해볼거예요'],
 'uris': None,
 'data': None,
 'metadatas': [{'source': 'mydata.txt'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

**add_text**

    텍스트를 임베딩하고 벡터 저장소에 추가

매개변수

    texts : 벡터 저장소에 추가할 텍스트 리스트
    metadatas : 메타데이터 리스트 (기본값 None)
    ids : 문서 ID 리스트 (기본값 None)
        입력하지 않으면 UUID를 사용해 자동으로 생성된다.
        기존의 id에 추가하면 덮어씌워지게 된다.

In [18]:
db.add_texts(
    ["이전에 추가한 문서를 덮어씌우기", "기존 거는 삭제가 됩니다."],
    metadatas=[{"source" : "mydata.txt"}, {"source" : "mydata.txt"}],
    ids=["1", "2"]
)

['1', '2']

In [19]:
db.get(["1"])

{'ids': ['1'],
 'embeddings': None,
 'documents': ['이전에 추가한 문서를 덮어씌우기'],
 'uris': None,
 'data': None,
 'metadatas': [{'source': 'mydata.txt'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

**delete()**

    - 벡터 저장소에 지정된 ID의 문서를 삭제

매개변수

    - ids : 삭제할 문서의 ID리스트

In [20]:
db.delete(ids=["1"])

In [22]:
db.get(["1", "2"])

{'ids': ['2'],
 'embeddings': None,
 'documents': ['기존 거는 삭제가 됩니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'source': 'mydata.txt'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

**reset_collection()**

    - 벡터 스토어의 컬렉션을 초기화한다.

In [23]:
# 초기화
db.reset_collection()

In [25]:
# 문서 조회
db.get()

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'data': None,
 'metadatas': [],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

### 벡터 저장소를 검색기로 변환
**as_retriever**
    
    벡터 저장소를 기반으로 VectorStoreRetriever를 생성한다.

매개변수

    **kwargs : 검색 함수에 전달할 키워드 인자
    search_type : 검색 유형 ("similarity", "mmr")
    search_kwargs : 검색 함수에 추가로 전달할 인자
        - k : 반환할 문서 수
        - score_thresold : 최소 유사도 임계값
        - fetch_k : MMR 알고리즘에 전달할 문서 수
        - lambda_mult : MMR 결과의 다양성 조절
        - filter : 문서 메타 데이터 필터링

In [26]:
db = Chroma.from_documents(
    documents=split_doc1 + split_doc2,
    embedding=OpenAIEmbeddings(),
    collection_name= "nlp"
)

In [27]:
# 검색기
retriever = db.as_retriever()

retriever.invoke("Word2Vec에 대해 알려줘")

[Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)\n\n정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이해 및 생성 작업에 사용됩니다.\n예시: OpenAI의 GPT 시리즈는 대표적인 대규모 언어 모델입니다.\n연관키워드: 자연어 처리, 딥러닝, 텍스트 생성\n\nFAISS (Facebook AI Similarity Search)\n\n정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source'),
 Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니다.\n예시: HuggingFace의 Transformers 라이브러리를 사용하여 감정 분석, 텍스트 생성 등의 작업을 수행할 수 있습니다.\n연관키워드: 자연어 처리, 딥러닝, 라이브러리\n\nDigital Transformation\n\n정의: 디지털 변환은 기술을 활용하여 


    k : 반환할 문서 수 (기본값 : 4)
    fetch_k : MMRK 알고리즘에 전달할 문서 수 (기본값 : 20)
    lambda_mult : MMR 결과의 다양성 조절 (0~1, 기본값 0.5, 0은 유사도만 고려, 1은 다양성만 고려)

In [ ]:
# 다양성이 높은 문서 검색
retriever = db.as_retriever(
    search_type="mmr", search_kwargs={"k" : 5, "lambda_mult" : 0.25, "fetch_k" : 10}
)

retriever.invoke("Word2Vec에 대해 알려줘")

[Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)\n\n정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이해 및 생성 작업에 사용됩니다.\n예시: OpenAI의 GPT 시리즈는 대표적인 대규모 언어 모델입니다.\n연관키워드: 자연어 처리, 딥러닝, 텍스트 생성\n\nFAISS (Facebook AI Similarity Search)\n\n정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source'),
 Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을 가집니다.\n연관키워드: 자연어 처리, 정보 검색, 데이터 마이닝\n\nDeep Learning\n\n정의: 딥러닝은 인공신경망을 이용하여 복잡한 문제를 해결하는 머신러닝의 한 분야입니다. 이는 데이터에서 고수준의

In [31]:
# 특정 임계값 이상의 유사도를 가진 문서만 탐색
retriever = db.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold" : 0.8}
)

retriever.invoke("Word2Vec에 대해 알려줘")

[Document(metadata={'source': './data/nlp-keywords.txt'}, page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)\n\n정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이해 및 생성 작업에 사용됩니다.\n예시: OpenAI의 GPT 시리즈는 대표적인 대규모 언어 모델입니다.\n연관키워드: 자연어 처리, 딥러닝, 텍스트 생성\n\nFAISS (Facebook AI Similarity Search)\n\n정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source')]

In [33]:
# 메타 데이터 필터 적용
retriever = db.as_retriever(
    search_kwargs={"filter" : {"source" : "./data/finance-keywords.txt"}, "k" : 2},
)

retriever.invoke("ESG에 대해 알려줘")

[Document(metadata={'source': './data/finance-keywords.txt'}, page_content='정의: ESG는 기업의 환경, 사회, 지배구조 측면을 고려하는 투자 접근 방식입니다.\n예시: S&P 500 ESG 지수는 우수한 ESG 성과를 보이는 기업들로 구성된 지수입니다.\n연관키워드: 지속가능 투자, 기업의 사회적 책임, 윤리 경영\n\nStock Buyback\n\n정의: 자사주 매입은 기업이 자사의 주식을 시장에서 다시 사들이는 것을 말합니다.\n예시: 애플은 S&P 500 기업 중 가장 큰 규모의 자사주 매입 프로그램을 운영하고 있습니다.\n연관키워드: 주주 가치, 자본 관리, 주가 부양\n\nCyclical Stocks\n\n정의: 경기순환주는 경제 상황에 따라 실적이 크게 변동하는 기업의 주식을 말합니다.\n예시: 포드, 제너럴 모터스와 같은 자동차 기업들은 S&P 500에 포함된 대표적인 경기순환주입니다.\n연관키워드: 경제 사이클, 섹터 분석, 투자 타이밍\n\nDefensive Stocks\n\n정의: 방어주는 경기 변동에 상관없이 안정적인 실적을 보이는 기업의 주식을 의미합니다.\n예시: 프록터앤갬블, 존슨앤존슨과 같은 생활필수품 기업들은 S&P 500 내 대표적인 방어주로 꼽힙니다.\n연관키워드: 안정적 수익, 저변동성, 리스크 관리'),
 Document(metadata={'source': './data/finance-keywords.txt'}, page_content='정의: 주식 리서치는 기업의 재무 상태, 사업 모델, 경쟁력 등을 분석하여 투자 의사 결정을 돕는 활동입니다.\n예시: 골드만삭스의 애널리스트들이 S&P 500 기업들에 대한 분기별 실적 전망을 발표했습니다.\n연관키워드: 투자 분석, 기업 가치평가, 시장 전망\n\nCorporate Governance\n\n정의: 기업 지배구조는 기업의 경영과 통제에 관한 시스템과 프로세스를 의미합니다.\n예시: S&P 500 기업들 중 이사회